# Importing Dependencies

In [2]:
import sys
!{sys.executable} -m pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python-4.6.0.66-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.9 MB)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
import os 
import shutil
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt 
import torch
import cv2
%matplotlib inline

/home/aswin/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
filenames = []

size_props = {
    'height':[],
    'width':[]
}

bounding_box_props = {
    'xmin':[],
    'ymin':[],
    'xmax':[],
    'ymax':[]
}

# Fetching required properties from annotations

In [6]:
annotations_path = './annotations'
for file in tqdm(os.listdir(annotations_path)):
    annotation = ET.parse(os.path.join(annotations_path, file))
    filenames.append(os.path.join(annotations_path, file))
    size = annotation.find('size')
    for name, prop_list in size_props.items():
        prop_value = size.find(name).text
        size_props[name].append(int(prop_value))
    bounding_box = annotation.find('object').find('bndbox')
    for name, prop_list in bounding_box_props.items():
        prop_value = bounding_box.find(name).text
        bounding_box_props[name].append(int(prop_value))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 433/433 [00:00<00:00, 20007.64it/s]


In [7]:
df = pd.DataFrame({
    'file':filenames,
    'width':size_props['width'],
    'height':size_props['height'],
    'xmin':bounding_box_props['xmin'],
    'ymin':bounding_box_props['ymin'],
    'xmax':bounding_box_props['xmax'],
    'ymax':bounding_box_props['ymax']
})

In [8]:
df.head()

,file,width,height,xmin,ymin,xmax,ymax
0,./annotations/Cars395.xml,400,301,241,182,288,205
1,./annotations/Cars275.xml,400,215,99,150,162,167
2,./annotations/Cars388.xml,400,300,149,130,255,158
3,./annotations/Cars167.xml,500,400,176,254,327,285
4,./annotations/Cars140.xml,435,290,283,239,424,286


# Making annotations compatible with YOLO

In [9]:
df['center_x'] = (df['xmax'] + df['xmin'])/(2*df['width'])
df['center_y'] = (df['ymax'] + df['ymin'])/(2*df['height'])

df['bb_width'] = (df['xmax'] - df['xmin'])/df['width']
df['bb_height'] = (df['ymax'] - df['ymin'])/df['height']

In [10]:
df.head()

,file,width,height,xmin,ymin,xmax,ymax,center_x,center_y,bb_width,bb_height
0,./annotations/Cars395.xml,400,301,241,182,288,205,0.661250,0.642857,0.117500,0.076412
1,./annotations/Cars275.xml,400,215,99,150,162,167,0.326250,0.737209,0.157500,0.079070
2,./annotations/Cars388.xml,400,300,149,130,255,158,0.505000,0.480000,0.265000,0.093333
3,./annotations/Cars167.xml,500,400,176,254,327,285,0.503000,0.673750,0.302000,0.077500
4,./annotations/Cars140.xml,435,290,283,239,424,286,0.812644,0.905172,0.324138,0.162069


In [11]:
# Keeping important columns only 
yolo_df = df[['file', 'center_x', 'center_y', 'bb_width', 'bb_height']]
# Performing 70-15-15 split
test_size = int(0.15 * len(df))

df_train, df_test = train_test_split(yolo_df, test_size=test_size)
df_train, df_val = train_test_split(df_train, test_size=test_size)

In [14]:
train_path = os.path.join('Images', 'train')
val_path = os.path.join('Images','val')
test_path = os.path.join('Images', 'test')
images_path = './images'

if not os.path.exists(train_path):
    os.makedirs(train_path)
    print('Made folder for train set')

if not os.path.exists(val_path):
    os.makedirs(val_path)
    print('Made folder for val set')

if not os.path.exists(test_path):
    os.makedirs(test_path)
    print('Made folder for test set')

# Creating train and test folders 

In [15]:
print('Moving images for train set')
for _, row  in tqdm(df_train.iterrows()):
    annotation_path = row['file']
    image_name = os.path.split(annotation_path)[-1].replace('.xml','')
    image_src = os.path.join(images_path, f'{image_name}.png')
    image_dst = os.path.join(train_path, f'{image_name}.png')
    shutil.copy2(image_src, image_dst)
    label_text = f"0 {row['center_x']} {row['center_y']} {row['bb_width']} {row['bb_height']}"
    with open(os.path.join(train_path, f'{image_name}.txt'), 'w') as f:
        f.write(label_text)
print('Done moving images for train set')

print('Moving images for val set')
for _, row  in tqdm(df_val.iterrows()):
    annotation_path = row['file']
    image_name = os.path.split(annotation_path)[-1].replace('.xml','')
    image_src = os.path.join(images_path, f'{image_name}.png')
    image_dst = os.path.join(val_path, f'{image_name}.png')
    shutil.copy2(image_src, image_dst)
    label_text = f"0 {row['center_x']} {row['center_y']} {row['bb_width']} {row['bb_height']}"
    with open(os.path.join(val_path, f'{image_name}.txt'), 'w') as f:
        f.write(label_text)
print('Done moving images for val set')

print('Moving images for test set')
for _, row  in tqdm(df_test.iterrows()):
    annotation_path = row['file']
    image_name = os.path.split(annotation_path)[-1].replace('.xml','')
    image_src = os.path.join(images_path, f'{image_name}.png')
    image_dst = os.path.join(test_path, f'{image_name}.png')
    shutil.copy2(image_src, image_dst)
    label_text = f"0 {row['center_x']} {row['center_y']} {row['bb_width']} {row['bb_height']}"
    with open(os.path.join(test_path, f'{image_name}.txt'), 'w') as f:
        f.write(label_text)
print('Done moving images for test set')

Moving images for train set


305it [00:00, 1302.42it/s]


Done moving images for train set
Moving images for val set


64it [00:00, 1855.55it/s]


Done moving images for val set
Moving images for test set


64it [00:00, 1689.72it/s]

Done moving images for test set


# Training YOLO model

In [16]:
# Cloning the ultralytics yolo repository
! git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 13181, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 13181 (delta 31), reused 27 (delta 13), pack-reused 13119
Receiving objects: 100% (13181/13181), 12.58 MiB | 3.29 MiB/s, done.
Resolving deltas: 100% (9042/9042), done.


In [17]:
# Installing the dependencies required for YOLO
! pip install -r yolov5/requirements.txt

  Using cached PyYAML-6.0-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (661 kB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached torch-1.12.0-cp39-cp39-manylinux1_x86_64.whl (776.3 MB)
     |████████████████████████████████| 19.1 MB 167 kB/s eta 0:00:01
  Using cached protobuf-3.20.1-cp39-cp39-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.0 MB)
  Using cached tensorboard-2.9.1-py3-none-any.whl (5.8 MB)
  Using cached seaborn-0.11.2-py3-none-any.whl (292 kB)
  Using cached ipython-8.4.0-py3-none-any.whl (750 kB)
  Using cached psutil-5.9.1-cp39-cp39-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (281 kB)
  Using cached charset_normalizer-2.1.0-py3-none-any.whl (39 kB)
     |████████████████████████████████| 139 kB 847 kB/s eta 0:00:01
  Using cached idna-3.3-py3-none-any.whl (61 kB)
  Using cached certifi-2022.6.15-py3-none-any.whl (160 kB)
  Using cached typing_extensions

In [18]:
# creating data.yaml file which will be used to train YOLO on custom data 
data = {
    'names':['License Plate'],
    'nc':1,
    'train':os.path.abspath(train_path),
    'val':os.path.abspath(val_path)
}

with open('data.yaml', 'w') as f:
    yaml.dump(data, f)

In [19]:
# Training the YOLO model 
! python ./yolov5/train.py --data ./data.yaml  --batch-size 8  --epochs 100 --weights yolov5/yolov5s.pt

train: weights=yolov5/yolov5s.pt, cfg=, data=./data.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
Command 'git fetch && git config --get remote.origin.url' timed out after 5 seconds
requirements: tensorboard>=2.4.1 not found and is required by YOLOv5, attempting auto-update...
You should consider upgrading via the '/home/aswin/Desktop/anpr/Plate_detector/env/bin/python -m pip install --upgrade pip' command.
  Using cached protobuf-3.19.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x8

Model summary: 270 layers, 7022326 parameters, 7022326 gradients, 15.9 GFLOPs

Transferred 343/349 items from yolov5/yolov5s.pt
AMP: checks passed ✅
Scaled weight_decay = 0.0005
optimizer: SGD with parameter groups 57 weight (no decay), 60 weight, 60 bias
train: Scanning '/home/aswin/Desktop/anpr/Plate_detector/Images/train' images an
train: New cache created: /home/aswin/Desktop/anpr/Plate_detector/Images/train.cache
val: Scanning '/home/aswin/Desktop/anpr/Plate_detector/Images/val' images and la
val: New cache created: /home/aswin/Desktop/anpr/Plate_detector/Images/val.cache
Plotting labels to yolov5/runs/train/exp/labels.jpg... 

AutoAnchor: 4.08 anchors/target, 1.000 Best Possible Recall (BPR). Current anchors are a good fit to dataset ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolov5/runs/train/exp
Starting training for 100 epochs...

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      0/99     1.72G    0.1052   0.02708 

     22/99     2.42G   0.03518  0.009691         0         3       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         64         64       0.88      0.859      0.876      0.471

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     23/99     2.42G   0.03495  0.009658         0         4       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         64         64      0.893      0.844      0.893      0.439

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     24/99     2.42G   0.03326  0.008557         0         1       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         64         64      0.895        0.8      0.881      0.453

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     25/99     2.42G   0.03471  0.008


     Epoch   gpu_mem       box       obj       cls    labels  img_size
     48/99     2.42G   0.02521  0.006967         0         3       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         64         64      0.919       0.89      0.931      0.506

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     49/99     2.42G   0.02573  0.007067         0         2       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         64         64      0.906      0.901       0.93      0.496

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     50/99     2.42G   0.02516  0.007065         0         2       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         64         64        0.9      0.922      0.932        0.5

     Epoch   gpu_mem       box      

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         64         64      0.891      0.875      0.907      0.473

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     74/99     2.42G   0.02073  0.006478         0         3       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         64         64      0.902      0.891      0.914      0.468

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     75/99     2.42G   0.01996   0.00567         0         1       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         64         64      0.903      0.891      0.915      0.479

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     76/99     2.42G   0.01952  0.005609         0         1       640: 100%|███
               Class     Images     L


     Epoch   gpu_mem       box       obj       cls    labels  img_size
     99/99     2.42G   0.01863  0.005709         0         2       640: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         64         64      0.905      0.889      0.899      0.482

100 epochs completed in 0.671 hours.
Optimizer stripped from yolov5/runs/train/exp/weights/last.pt, 14.5MB
Optimizer stripped from yolov5/runs/train/exp/weights/best.pt, 14.5MB

Validating yolov5/runs/train/exp/weights/best.pt...
Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         64         64      0.883      0.859      0.912      0.514
Results saved to yolov5/runs/train/exp


# Loading best model 

In [3]:
# Fetching the latest runs
yolo_path = './yolov5/runs/train/'
latest_run = os.listdir(yolo_path)[-1]

# Fetching the best weights 
best_weights = os.path.join(yolo_path, latest_run, 'weights', 'best.pt')

# Loading the model with best weights trained on custom data 
model = torch.hub.load('ultralytics/yolov5', 'custom', best_weights)

Using cache found in /home/aswin/.cache/torch/hub/ultralytics_yolov5_master
requirements: torchvision>=0.8.1 not found and is required by YOLOv5, attempting auto-update...
You should consider upgrading via the '/home/aswin/Desktop/anpr/Plate_detector/env/bin/python -m pip install --upgrade pip' command.

requirements: protobuf<=3.20.1 not found and is required by YOLOv5, attempting auto-update...
You should consider upgrading via the '/home/aswin/Desktop/anpr/Plate_detector/env/bin/python -m pip install --upgrade pip' command.

requirements: 2 packages updated per /home/aswin/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-7-29 Python-3.9.1 torch-1.11.0+cu102 CUDA:0 (GeForce GTX 1050, 4042MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


# Visualizing predictions

In [4]:
%matplotlib inline
# Fetching random 16 images from test set
test_files = df_test['file'].apply(lambda x: x.replace('annotations','images').replace('xml','png'))
test_images = np.random.choice(test_files, size=(4,4))

# Visualizing predictions
fig, ax = plt.subplots(figsize=(16,16), nrows=4, ncols=4)
for i in range(4):
    for j in range(4):
        # Reading the image 
        image = cv2.imread(test_images[i,j])
        # Convert image to RGB colorspace
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Predicting from model
        results = model(image)
        # Getting co ordinates of license plate
        results_df = results.pandas().xyxy[0].loc[0]
        x_min = int(results_df['xmin'])
        x_max = int(results_df['xmax'])
        y_min = int(results_df['ymin'])
        y_max = int(results_df['ymax'])
        # Cropping license plate from image ""
        number_plate = image[y_min:y_max,x_min:x_max]
        # Performing OCR on image 
        text = pt.image_to_string(number_plate)
        # Showing the results 
        ax[i,j].set_title(str(text).strip())
        ax[i,j].imshow(np.squeeze(results.render()))
plt.show()

NameError: name 'df_test' is not defined

I have created a repository on github which uses the model trained in this notebook to perform real time license plate detection, then read the license plate numbers using OCR and then make an entry in a database along with the timestamp when the license plate was detected. 

In this way, we will be able to keep a track of unknown vehicles in the society which may cause inconvinience to the society members.

I have also created a web UI where the society residents can register themselves and their vehicles 

<a href="https://github.com/aayush1036/residential-safety-solution">Github project</a>

Please star and fork the repository if you like the project

In [28]:
latest_run

'exp'